In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
import warnings
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from pmdarima import auto_arima
# Ignore SettingWithCopyWarning
warnings.filterwarnings("ignore")
# Assuming you have a hierarchical DataFrame named 'sales_data' with columns 'Foodcourt', 'Restaurant', 'Item'
df = pd.read_excel("meals_std.xlsx")
df = df.reset_index(drop = True)
df = df[["foodcourt","restaurant","restaurantmenuitem","date","total_count"]]
len(df)

112196

In [18]:
output = pd.read_excel("output_summary_7.xlsx")
output.columns


Index(['foodcourt', 'restaurant', 'restaurantmenuitem', 'accuracy_xgb_lag',
       'accuracy_xgb_norm', 'accuracy_rf_lag', 'accuracy_prophet',
       'accuracy_prophet_multi', 'accuracy_deep_AR', 'accuracy_lgb_lag',
       'accuracy_deepar_multi', 'accuracy_lr', 'accuracy_xgb_multi'],
      dtype='object')

In [19]:
output['max_accuracy'] = output[['accuracy_xgb_lag','accuracy_rf_lag',"accuracy_lgb_lag", 'accuracy_xgb_norm','accuracy_prophet',"accuracy_prophet_multi","accuracy_deep_AR","accuracy_deepar_multi",'accuracy_lr', 'accuracy_xgb_multi']].max(axis=1)
output['best_model'] = output[['accuracy_xgb_lag', 'accuracy_rf_lag',"accuracy_lgb_lag",'accuracy_xgb_norm','accuracy_prophet',"accuracy_prophet_multi","accuracy_deep_AR","accuracy_deepar_multi",'accuracy_lr', 'accuracy_xgb_multi']].idxmax(axis=1)

In [20]:
def segment_max_accuracy(accuracy):
    if accuracy >= 0.90:
        return '>90'
    elif .80 <= accuracy < .90:
        return '80-90'
    elif .70 <= accuracy < .80:
        return '70-80'
    elif .50 <= accuracy < .70:
        return '50-70'
    else:
        return '<50'

def add_max_accuracy_segment(df):
    df['max_accuracy_segment'] = df['max_accuracy'].apply(segment_max_accuracy)
    return df

# Apply the function to your DataFrame
output = add_max_accuracy_segment(output)

In [21]:
pivot_table_df = output.pivot_table(index='max_accuracy_segment',  aggfunc='size', fill_value=0)
desired_order_rows = ['>90','80-90','70-80','50-70','<50' ]
pivot_table_df = pivot_table_df.reindex(index=desired_order_rows)
pivot_table_df

max_accuracy_segment
>90      240
80-90    271
70-80    256
50-70    162
<50       44
dtype: int64